In [ ]:
import sys
import subprocess
import importlib
import threading
import socket
import time
import struct

# 전역 변수 설정
HOST_CAM = '192.168.0.60'
PORT_CAM = 80
PORT_MOT = 81

# 1. 패키지 자동 설치 함수
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"{package_name} installation completed")
        except subprocess.CalledProcessError as e:
            print(f"{package_name} installation failed (exit code {e.returncode})")
            sys.exit(1)

print("Checking required packages...")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("PyQt5")

import numpy as np
import cv2
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5 import QtWidgets, QtGui

# ---------------------------------------------------------
# 2. 윈도우 소켓 버그 수정용 함수
# ---------------------------------------------------------
def recvall(sock, count):
    buf = b''
    while len(buf) < count:
        try:
            newbuf = sock.recv(count - len(buf))
            if not newbuf: return None
            buf += newbuf
        except socket.timeout:
            raise
        except BlockingIOError:
            continue
    return buf

client_cam = None
client_mot = None

current_command = -1 
running = False 
label_widget = None 
camera_thread = None


# ---------------------------------------------------------
# 소켓 연결
# ---------------------------------------------------------
def try_connect_esp32(ip_address):
    global client_cam, client_mot
    print(f"Connecting to ESP32 ({ip_address})...")
    
    if client_cam:
        try: client_cam.close()
        except: pass
    if client_mot:
        try: client_mot.close()
        except: pass

    try:
        cam_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        mot_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        mot_sock.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        
        cam_sock.settimeout(3)
        mot_sock.settimeout(3)

        cam_sock.connect((ip_address, PORT_CAM))
        mot_sock.connect((ip_address, PORT_MOT))
        
        cam_sock.settimeout(120)
        mot_sock.settimeout(120)
        
        client_cam = cam_sock
        client_mot = mot_sock
        print("Connected successfully!")
        return True
        
    except Exception as e:
        print(f"Connection Failed: {e}")
        return False

# ---------------------------------------------------------
# 카메라 쓰레드
# ---------------------------------------------------------
def camMain():
    global current_command, running
    
    t_prev = time.time()
    cnt_frame = 0
    
    DISPLAY_WIDTH = 640
    DISPLAY_HEIGHT = 480
    
    while label_widget is None and running:
        time.sleep(0.1)
        
    try:
        if label_widget: label_widget.resize(DISPLAY_WIDTH, DISPLAY_HEIGHT)
    except: pass

    while running:
        if client_cam is None:
            time.sleep(1)
            continue

        try:
            # 영상 요청
            cmd = 12
            cmd = struct.pack('B', cmd)
            client_cam.sendall(cmd) 

            # 헤더 수신
            data_len_bytes = recvall(client_cam, 4)
            if not data_len_bytes: continue
            data_len = struct.unpack('I', data_len_bytes)[0]
            
            # 데이터 수신
            data = recvall(client_cam, data_len)
            if not data: continue

            # 이미지 디코딩
            np_data = np.frombuffer(data, dtype='uint8')
            frame = cv2.imdecode(np_data, 1)
            if frame is None: continue
            
            frame = cv2.rotate(frame, cv2.ROTATE_180) 
            
            # 리사이즈
            frame_resized = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT), interpolation=cv2.INTER_LINEAR)
            
            h, w, c = frame_resized.shape
            qImg = QtGui.QImage(frame_resized.data, w, h, w*c, QtGui.QImage.Format_RGB888)
            pixmap = QtGui.QPixmap.fromImage(qImg.rgbSwapped())
            
            if label_widget is not None:
                label_widget.setPixmap(pixmap)

            cnt_frame += 1
            t_now = time.time()
            if t_now - t_prev >= 1.0:
                cmd_names = ["Forward", "Right", "Left", "Stop"]
                cmd_str = "Idle"
                if 0 <= current_command < len(cmd_names):
                    cmd_str = cmd_names[current_command]
                
                print(f"FPS: {cnt_frame}, Direction: {cmd_str} ({current_command})")
                
                cnt_frame = 0
                t_prev = t_now
                
        except socket.timeout:
            print("Socket timed out. Waiting...")
            time.sleep(0.5) 
            continue
        except Exception as e:
            print(f"Cam Error: {e}")
            break

# ---------------------------------------------------------
# 모터 제어
# ---------------------------------------------------------
def send_motor_command(cmd_idx):
    global current_command
    current_command = cmd_idx

def motorMain():
    global current_command, running, client_mot
    
    while running:
        if client_mot:
            try:
                cmd_char = b'S\n'
                
                if current_command == 0: cmd_char = b'F\n'   # Forward
                elif current_command == 1: cmd_char = b'R\n' # Right
                elif current_command == 2: cmd_char = b'L\n' # Left
                elif current_command == 3: cmd_char = b'S\n' # Stop
                elif current_command == 4: cmd_char = b'B\n' # Backward
                
                client_mot.sendall(cmd_char)
            except:
                pass
        
        time.sleep(0.1)
    
# ---------------------------------------------------------
# 메인 윈도우
# ---------------------------------------------------------
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('RC Car Controller (Drive Only)')
        self.setGeometry(100, 100, 350, 600) # 버튼 추가로 창 높이 증가
        self.setFocusPolicy(Qt.StrongFocus)

        cw = QWidget()
        self.setCentralWidget(cw)
        layout = QVBoxLayout()
        cw.setLayout(layout)

        # 1. 화면
        global label_widget
        label_widget = QLabel("Click Here to Focus")
        label_widget.setAlignment(Qt.AlignCenter)
        label_widget.setStyleSheet("background-color: black; color: white;")
        label_widget.setScaledContents(True)
        layout.addWidget(label_widget)

        # 2. 버튼
        btn_layout = QGridLayout()
        layout.addLayout(btn_layout)

        self.btn_fwd = QPushButton("Forward (W)")
        self.btn_left = QPushButton("Left (A)")
        self.btn_right = QPushButton("Right (D)")
        self.btn_stop = QPushButton("Stop (Space)") # 중앙 정지 버튼
        self.btn_back = QPushButton("Backward (S)") # 후진 버튼 추가
        
        for btn in [self.btn_fwd, self.btn_left, self.btn_right, self.btn_stop, self.btn_back]:
            btn.setFixedHeight(50)
            btn.setStyleSheet("font-weight: bold; font-size: 14px;")
            btn.setFocusPolicy(Qt.NoFocus)

        btn_layout.addWidget(self.btn_fwd, 0, 1)   # 상
        btn_layout.addWidget(self.btn_left, 1, 0)  # 좌
        btn_layout.addWidget(self.btn_stop, 1, 1)  # 중 (정지)
        btn_layout.addWidget(self.btn_right, 1, 2) # 우
        btn_layout.addWidget(self.btn_back, 2, 1)  # 하 (후진)

        # 3. IP 변경 버튼
        self.btn_change_ip = QPushButton(f"Change IP Address (Current: {HOST_CAM})")
        self.btn_change_ip.setFixedHeight(40)
        self.btn_change_ip.clicked.connect(self.on_change_ip_clicked)
        self.btn_change_ip.setFocusPolicy(Qt.NoFocus)
        layout.addWidget(self.btn_change_ip)

        self.btn_fwd.pressed.connect(lambda: send_motor_command(0))
        self.btn_left.pressed.connect(lambda: send_motor_command(2))
        self.btn_right.pressed.connect(lambda: send_motor_command(1))
        self.btn_stop.pressed.connect(lambda: send_motor_command(3))
        self.btn_back.pressed.connect(lambda: send_motor_command(4)) # 후진(4)

        self.btn_fwd.released.connect(lambda: send_motor_command(-1))
        self.btn_left.released.connect(lambda: send_motor_command(-1))
        self.btn_right.released.connect(lambda: send_motor_command(-1))
        self.btn_stop.released.connect(lambda: send_motor_command(-1))
        self.btn_back.released.connect(lambda: send_motor_command(-1))

        info_label = QLabel("Controls:\n- WASD / Arrows: Move\n- Space: Stop")
        layout.addWidget(info_label)

        self.setFocus()

    def focusNextPrevChild(self, next):
        return False

    def on_change_ip_clicked(self):
        global HOST_CAM, running, camera_thread, motor_thread
        text, ok = QInputDialog.getText(self, 'Change IP', 'Enter ESP32 IP Address:', text=HOST_CAM)
        if ok and text:
            print("Reconnecting to new IP...")
            HOST_CAM = text
            self.btn_change_ip.setText(f"Change IP Address (Current: {HOST_CAM})")
            
            running = False
            if camera_thread: camera_thread.join(timeout=1.0)
            
            if try_connect_esp32(HOST_CAM):
                QMessageBox.information(self, "Success", "Connected to new IP!")
                running = True
                
                camera_thread = threading.Thread(target=camMain)
                camera_thread.setDaemon(True)
                camera_thread.start()
                
                if 'motor_thread' in globals() and motor_thread: # motorMain이 정의된 경우
                     motor_thread = threading.Thread(target=motorMain)
                     motor_thread.setDaemon(True)
                     motor_thread.start()
            else:
                QMessageBox.warning(self, "Failed", "Could not connect.")
            
            self.setFocus()

    def keyPressEvent(self, event):
        key = event.key()
        if key == Qt.Key_Escape:
            self.close()
            return

        if not event.isAutoRepeat(): 
            if key == Qt.Key_W or key == Qt.Key_Up:
                self.btn_fwd.setDown(True)
                send_motor_command(0)
            elif key == Qt.Key_A or key == Qt.Key_Left:
                self.btn_left.setDown(True)
                send_motor_command(2)
            elif key == Qt.Key_D or key == Qt.Key_Right:
                self.btn_right.setDown(True)
                send_motor_command(1)
            elif key == Qt.Key_S or key == Qt.Key_Down:
                self.btn_back.setDown(True) 
                send_motor_command(4)  
            elif key == Qt.Key_Space:
                self.btn_stop.setDown(True)
                send_motor_command(3)

    def keyReleaseEvent(self, event):
        key = event.key()
        if not event.isAutoRepeat():
            if key in [Qt.Key_W, Qt.Key_Up, Qt.Key_A, Qt.Key_Left, 
                       Qt.Key_D, Qt.Key_Right, Qt.Key_S, Qt.Key_Down, Qt.Key_Space]:
                
                self.btn_fwd.setDown(False)
                self.btn_left.setDown(False)
                self.btn_right.setDown(False)
                self.btn_stop.setDown(False)
                self.btn_back.setDown(False)
                send_motor_command(-1)

    def closeEvent(self, event):
        print("Window closing...")
        event.accept()

# ---------------------------------------------------------
# 실행부
# ---------------------------------------------------------
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)

connected = False
while not connected:
    connected = try_connect_esp32(HOST_CAM)
    
    if not connected:
        text, ok = QInputDialog.getText(None, 'Connection Failed', 
                                        'Failed to connect.\nEnter ESP32 IP Address:', 
                                        text=HOST_CAM)
        if ok and text:
            HOST_CAM = text 
        else:
            print("User cancelled connection.")
            sys.exit()

if connected:
    mw = MainWindow()
    mw.show()

    running = True
    camera_thread = threading.Thread(target=camMain)
    camera_thread.setDaemon(True) 
    camera_thread.start()

    motor_thread = threading.Thread(target=motorMain)
    motor_thread.setDaemon(True)
    motor_thread.start()

    print("Drive Mode Running. Press ESC to quit.")
    
    try:
        app.exec_()
    except SystemExit:
        pass
    finally:
        print("Cleaning up resources...")
        running = False
        try: client_cam.close() 
        except: pass
        try: client_mot.close() 
        except: pass
        try: del mw
        except: pass
        try: del app
        except: pass
        print("Done.")